In [1]:
import sys
print(sys.executable)
print(sys.prefix)


G:\GeekDS\venv\python.exe
G:\GeekDS\venv


In [2]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [3]:
!git remote -v

origin	https://github.com/Lavishgangwani/DataScienceGeek.git (fetch)
origin	https://github.com/Lavishgangwani/DataScienceGeek.git (push)


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split , GridSearchCV , RandomizedSearchCV
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

## Used Car Price Prediction
1) Problem statement.
    - This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
    - If user can predict the price of the car based on input features.
    - Prediction results can be used to give new seller the price suggestion based on market condition.
2) Data Collection.
    - The Dataset is collected from scrapping from cardheko webiste
    - The data consists of 13 column and 15411 rows.

In [5]:
#loading the data
df = pd.read_csv('cardekho_imputated.csv' , index_col=[0])
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [6]:
df.shape

(15411, 13)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15411 entries, 0 to 19543
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_name           15411 non-null  object 
 1   brand              15411 non-null  object 
 2   model              15411 non-null  object 
 3   vehicle_age        15411 non-null  int64  
 4   km_driven          15411 non-null  int64  
 5   seller_type        15411 non-null  object 
 6   fuel_type          15411 non-null  object 
 7   transmission_type  15411 non-null  object 
 8   mileage            15411 non-null  float64
 9   engine             15411 non-null  int64  
 10  max_power          15411 non-null  float64
 11  seats              15411 non-null  int64  
 12  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1.6+ MB


In [8]:
#checking null values
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [9]:
#drop unnecessary cols
df.drop(columns=['car_name' , 'brand'] , axis=1 , inplace = True)
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [10]:
cat_cols = df.select_dtypes(include='O')
for i in cat_cols.columns:
    print(f'Value Counts of {i}')
    print(cat_cols[i].value_counts())
    print('-----'*10)

Value Counts of model
model
i20            906
Swift Dzire    890
Swift          781
Alto           778
City           757
              ... 
Ghibli           1
Altroz           1
GTC4Lusso        1
Aura             1
Gurkha           1
Name: count, Length: 120, dtype: int64
--------------------------------------------------
Value Counts of seller_type
seller_type
Dealer              9539
Individual          5699
Trustmark Dealer     173
Name: count, dtype: int64
--------------------------------------------------
Value Counts of fuel_type
fuel_type
Petrol      7643
Diesel      7419
CNG          301
LPG           44
Electric       4
Name: count, dtype: int64
--------------------------------------------------
Value Counts of transmission_type
transmission_type
Manual       12225
Automatic     3186
Name: count, dtype: int64
--------------------------------------------------


In [11]:
#now extraction columns types
cat_features = [i for i in df.columns if df[i].dtypes == 'O']
print("Categorical Features : ",len(cat_features))

num_features = [i for i in df.columns if df[i].dtypes != 'O']
print("Numerical Features : ",len(num_features))

descrete_num_features = [i for i in num_features if df[i].nunique() <= 25] 
print("Descrete Numerical Features : ",len(descrete_num_features))

continous_num_features = [i for i in num_features if df[i].nunique() >= 25]
print("Continuous Numerical Features : ",len(continous_num_features))

Categorical Features :  4
Numerical Features :  7
Descrete Numerical Features :  2
Continuous Numerical Features :  5


In [12]:
#splitt the data
X = df.drop(columns=['selling_price'] , axis=1)
y = df['selling_price']

### Feature Encoding and Scaling
- __One Hot Encoding for Columns which had lesser unique values and not ordinal__

- __One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.__

In [13]:
le = LabelEncoder()
X['model']= le.fit_transform(X['model'])

In [14]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [15]:
# Create Column Transformer with 3 types of transformers
cat_feat = X.select_dtypes(include='object').columns
num_feat = X.select_dtypes(exclude='object').columns

st = StandardScaler()
ohe_ = OneHotEncoder()

from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('OHE',ohe_,cat_feat),
    ('StScaler',st,num_feat)
],remainder='passthrough')

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('OHE', OneHotEncoder(),
                                 Index(['seller_type', 'fuel_type', 'transmission_type'], dtype='object')),
                                ('StScaler', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])

In [16]:
#now preprocess the data
X = preprocessor.fit_transform(X)
X

array([[ 0.        ,  1.        ,  0.        , ..., -1.32425883,
        -1.26335238, -0.40302241],
       [ 0.        ,  1.        ,  0.        , ..., -0.55471774,
        -0.43257082, -0.40302241],
       [ 0.        ,  1.        ,  0.        , ..., -0.55471774,
        -0.47911321, -0.40302241],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.02291783,
         0.06822523, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ...,  1.32979434,
         0.91715831,  2.07344426],
       [ 1.        ,  0.        ,  0.        , ...,  0.02099878,
         0.39588361, -0.40302241]])

In [17]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [18]:
#split the data into train and test
X_train , X_test , y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [19]:
X_train.shape , X_test.shape

((12328, 17), (3083, 17))

## Model Building and Evaluation

In [21]:
def metrics_scored(true,predicted):
    r2 = r2_score(true,predicted)
    mse = mean_squared_error(true,predicted)
    mae = mean_absolute_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true , predicted))

    return r2 , mse , mae , rmse

In [23]:
#model building


models = {
    "Random Forest" : RandomForestRegressor()
}



for i in range(len(list(models))):
    algo = list(models.values())[i]
    algo.fit(X_train , y_train)

    #make Prediction
    y_pred_train = algo.predict(X_train)
    y_pred_test = algo.predict(X_test)

    #metrics evaluation
    
    #for train data
    r2_score_train , mean_sq_train , mean_abs_train , root_mean_sq_train = metrics_scored(y_train , y_pred_train)

    #for test data
    r2_score_test , mean_sq_test , mean_abs_test , root_mean_sq_test = metrics_scored(y_test , y_pred_test)

    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(root_mean_sq_train))
    print("- Mean Squared Error: {:.4f}".format(mean_sq_train))
    print("- Mean Absolute Error: {:.4f}".format(mean_abs_train))
    print("- R2 Score: {:.4f}".format(r2_score_train))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(root_mean_sq_test))
    print("- Mean Squared Error: {:.4f}".format(mean_sq_test))
    print("- Mean Absolute Error: {:.4f}".format(mean_abs_test))
    print("- R2 Score: {:.4f}".format(r2_score_test))
    
    print('='*35)
    print('\n')

Random Forest
Model performance for Training set
- Root Mean Squared Error: 127347.6159
- Mean Squared Error: 16217415283.6740
- Mean Absolute Error: 39908.1773
- R2 Score: 0.9805
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 390679.7952
- Mean Squared Error: 152630702407.8079
- Mean Absolute Error: 104306.9373
- R2 Score: 0.7698




## Hyperparameter Tuning

In [24]:
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

In [25]:
rs_models = [(
    "RF" , RandomForestRegressor(),rf_params
)]

In [27]:
model_params = {}

for name , model , params in rs_models:
    rs = RandomizedSearchCV(estimator=model,
                           param_distributions=rf_params,
                           n_iter=100,
                           scoring='r2',
                           verbose=2,
                           n_jobs=-1,
                           cv=3)

    rs.fit(X_train,y_train)
    model_params[name] = {'best_params': rs.best_params_, 'best_score': rs.best_score_}
    
for model_name, results in model_params.items():
    print(f"---------------- Best Params for {model_name} -------------------")
    print(f"Best Parameters: {results['best_params']}")
    print(f"Best Score: {results['best_score']}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


G:\GeekDS\venv\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
87 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "G:\GeekDS\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "G:\GeekDS\venv\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "G:\GeekDS\venv\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "G:\GeekDS\venv\lib\site-packages\sklearn\utils\_param_validation.py", line 95, in 

---------------- Best Params for RF -------------------
Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'max_features': 8, 'max_depth': 15}
Best Score: 0.8812571534561567
